## Import libraries.

In [40]:
import os                           # For file processing.
import csv                          # For reading csv files.
import shutil
from keras.models import Model
#from keras.optimizers import Adam
from keras.optimizers import adam_v2

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np

In [ ]:
from google.colab import drive
drive.mount ('/content/gdrive')

!unzip gdrive/My\ Drive/data/archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/5/00005_00053_00010.png  
  inflating: train/5/00005_00053_00011.png  
  inflating: train/5/00005_00053_00012.png  
  inflating: train/5/00005_00053_00013.png  
  inflating: train/5/00005_00053_00014.png  
  inflating: train/5/00005_00053_00015.png  
  inflating: train/5/00005_00053_00016.png  
  inflating: train/5/00005_00053_00017.png  
  inflating: train/5/00005_00053_00018.png  
  inflating: train/5/00005_00053_00019.png  
  inflating: train/5/00005_00053_00020.png  
  inflating: train/5/00005_00053_00021.png  
  inflating: train/5/00005_00053_00022.png  
  inflating: train/5/00005_00053_00023.png  
  inflating: train/5/00005_00053_00024.png  
  inflating: train/5/00005_00053_00025.png  
  inflating: train/5/00005_00053_00026.png  
  inflating: train/5/00005_00053_00027.png  
  inflating: train/5/00005_00053_00028.png  
  inflating: train/5/00005_00053_00029.png  
  inflating: train/5/00005_00054_00000.png  
  in

In [ ]:
#Set directory values
#split_size=0.1
#print('split_size =',split_size)

# Get the current directory.
current_dir = os.getcwd()

# the data will be written to a working directory.
working_dir    = os.path.join(current_dir,'data')
train_data_dir = os.path.join(current_dir,'Train')
test_data_dir  = os.path.join(current_dir,'Test')
meta_data_dir  = os.path.join(current_dir,'Meta')

print('current_dir      = ',current_dir)
print('working_dir      = ',working_dir)
print('train_data_dir   = ',train_data_dir)
print('test_data_dir    = ',test_data_dir)
print('meta_data_dir    = ',meta_data_dir)
meta_csv_file = (os.path.join(current_dir,'Meta.csv'))
print('meta_csv_file    = ',meta_csv_file)


current_dir      =  /content
working_dir      =  /content/data
train_data_dir   =  /content/Train
test_data_dir    =  /content/Test
meta_data_dir    =  /content/Meta
meta_csv_file    =  /content/Meta.csv


### To properly reference the 43 classes in teh metadata, create a directory structure to allow the data generators to match the classes to the test, validation and training datasets.

In [ ]:
with open(meta_csv_file,"r") as csvfile:
    r= csv.reader(csvfile,delimiter =',')
    for i,row in enumerate(r):
        if i == 0: 
            continue
        label = row[1]
        print('label',label)
        img_name = label + '.png'
        print('img_name',img_name)

        # Creayte the sub-folder if it does not exist
        dest = os.path.join(meta_data_dir,label)
        if not os.path.isdir(dest):
            os.makedirs(dest)
        # Copy files to the relevant working directory   
        to_move = os.path.join(meta_data_dir,img_name)
        print('to_move',to_move)
        print('dest',dest)
        shutil.move(to_move,dest)

label 27
img_name 27.png
to_move /content/Meta/27.png
dest /content/Meta/27
label 0
img_name 0.png
to_move /content/Meta/0.png
dest /content/Meta/0
label 1
img_name 1.png
to_move /content/Meta/1.png
dest /content/Meta/1
label 10
img_name 10.png
to_move /content/Meta/10.png
dest /content/Meta/10
label 11
img_name 11.png
to_move /content/Meta/11.png
dest /content/Meta/11
label 12
img_name 12.png
to_move /content/Meta/12.png
dest /content/Meta/12
label 13
img_name 13.png
to_move /content/Meta/13.png
dest /content/Meta/13
label 14
img_name 14.png
to_move /content/Meta/14.png
dest /content/Meta/14
label 15
img_name 15.png
to_move /content/Meta/15.png
dest /content/Meta/15
label 16
img_name 16.png
to_move /content/Meta/16.png
dest /content/Meta/16
label 17
img_name 17.png
to_move /content/Meta/17.png
dest /content/Meta/17
label 18
img_name 18.png
to_move /content/Meta/18.png
dest /content/Meta/18
label 19
img_name 19.png
to_move /content/Meta/19.png
dest /content/Meta/19
label 2
img_name 2.p

In [ ]:
class_subset = sorted(os.listdir(meta_data_dir))[:43] # Using only the first 10 classes
print('class_subset    = ',class_subset)

class_subset    =  ['.~lock.ClassesInformation.ods#', '.~lock.ClassesInformationStrong.ods#', '0', '1', '1.1', '1.13', '1.2', '1.22', '1.24', '1.3.2', '1.32', '1.33', '1.34', '1.36', '1.37', '1.39', '1.5.2', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '2.1', '2.2', '2.3', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '3.1']


### Create a training data generator to resize the images to the VGG16 expected size and to split the traininf data into:
  -  A training set
  -  A validation set

Use an 85% to 15% split for now.

In [ ]:
BATCH_SIZE = 64
# The default input size for this model is 224x224.
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
dim = (IMAGE_WIDTH, IMAGE_HEIGHT)

train_generator = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing

### Create a training data generator to resize the images to the VGG16 expected size and to split the traininf data into:

In [ ]:
print(train_data_dir)
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

/content/Train
Found 19590 images belonging to 43 classes.


### Create a training data generator to resize the images to the VGG16 expected size and to split the traininf data into:

In [ ]:
validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

Found 3450 images belonging to 43 classes.


In [ ]:
### Create a test dataset.
test_csv_file = (os.path.join(current_dir,'Test.csv'))
print('test_csv_file    = ',test_csv_file)


test_csv_file    =  /content/Test.csv


In [ ]:
#prepare_test(source_test_folder,test_csv_file)
with open(test_csv_file,"r") as csvfile:
    r= csv.reader(csvfile,delimiter =',')
    for i,row in enumerate(r):
        if i == 0: 
            continue
        label = row[-2]
        print('label =',label)
        img_name = row[-1]
        print('img_name =',img_name)
        # Creayte the sub-folder if it does not exist
        print('test_data_dir =',test_data_dir)
        dest = os.path.join(test_data_dir,label)
        if not os.path.isdir(dest):
            os.makedirs(dest)
        print('dest          =',dest)

        # Copy files to the relevant working directory   
        to_move = os.path.join(test_data_dir,img_name)
        print('to_move       =',to_move)
        #shutil.move(to_move,dest)

Streaming output truncated to the last 5000 lines.
label = 8
img_name = Test/11630.png
test_data_dir = /content/Test
dest          = /content/Test/8
to_move       = /content/Test/Test/11630.png
label = 7
img_name = Test/11631.png
test_data_dir = /content/Test
dest          = /content/Test/7
to_move       = /content/Test/Test/11631.png
label = 2
img_name = Test/11632.png
test_data_dir = /content/Test
dest          = /content/Test/2
to_move       = /content/Test/Test/11632.png
label = 38
img_name = Test/11633.png
test_data_dir = /content/Test
dest          = /content/Test/38
to_move       = /content/Test/Test/11633.png
label = 30
img_name = Test/11634.png
test_data_dir = /content/Test
dest          = /content/Test/30
to_move       = /content/Test/Test/11634.png
label = 10
img_name = Test/11635.png
test_data_dir = /content/Test
dest          = /content/Test/10
to_move       = /content/Test/Test/11635.png
label = 2
img_name = Test/11636.png
test_data_dir = /content/Test
dest          = /co

In [ ]:
testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Found 12630 images belonging to 43 classes.


Using Pre-trained Layers for Feature Extraction
In this section, we'll demonstrate how to perform Transfer Learning without fine-tuning the pre-trained layers. Instead, we'll first use pre-trained layers to process our image dataset and extract visual features for prediction. Then we are creating a Fully-connected layer and Output layer for our image dataset. Finally, we will train these layers with backpropagation.